<div align='center'><font size="5">Financial Statement Correlation Data Analysis</font></div>
<!-- <div align='center'><font size="4">Ibnu Muhammad</font></div> -->
<br>
<hr>

<div><font size=3>Hi guys! In this notebook, I share my perspective and way of evaluating commodities price in some statistic manners to allocate portions of my wealth by converting them to assets. In the mean of time, I still elaborate and evaluate my model to fit the actual market. So bear with me in couple future of weeks or months guys!
<br>
<br>
<div><font size=3>Feel free to reach me out in the comments section or my linkedin <a href="https://www.linkedin.com/in/ibnummuhammad">https://www.linkedin.com/in/ibnummuhammad</a> if you'd like to discuss anything :)
<br>
<br>
<div><font size=3>And last but not least, I automate the data collection. Check out my github for the data scraping code <a href="https://github.com/ibnummuhammad/financial-data-scraping">https://github.com/ibnummuhammad/financial-data-scraping</a> !

In [ ]:
import os
import re
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
df = {}

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        df[filename] = pd.read_csv(os.path.join(dirname, filename))

In here, I use a naive Moving Average Model to evaluate current actual gold price whether it is over-value or under-value by looking in perspective of windowed time-range which I think is quite simple yet powerful. Though, this model still can't see how other commodities or public sentiment affecting the price.

In [ ]:
profit_pct = 0.05

df['pluang-gold-daily-dataset.csv']['updated_at'] = pd.to_datetime(df['pluang-gold-daily-dataset.csv']['updated_at'])
df_gold_pivot = df['pluang-gold-daily-dataset.csv'].pivot(index='updated_at', columns='quantity (gram)', values=['sell','buy'])
df_gold_ma = df_gold_pivot['sell'].rolling(window=60).mean()
df_gold_ma = df_gold_ma.rename(columns={1: "Sell (MA - 2 Months)"})
df_gold_ma_yearly = df_gold_pivot['sell'].rolling(window=365).mean()
df_gold_ma_yearly = df_gold_ma_yearly.rename(columns={1: "Sell (MA - Yearly)"})
df_gold_concat = pd.concat([df_gold_pivot, df_gold_ma, df_gold_ma_yearly], axis=1)

fig = go.Figure([
    go.Scatter(
        name='MA 1y',
        x=df_gold_concat.index,
        y=df_gold_concat.iloc[:, 3],
        mode='lines',
        line=dict(color='rgb(255,127,80)'),
    ),
    go.Scatter(
        name='MA 2m',
        x=df_gold_concat.index,
        y=df_gold_concat.iloc[:, 2],
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
    ),
    go.Scatter(
        name='MA 2m (upper)',
        x=df_gold_concat.index,
        y=df_gold_concat.iloc[:, 2]+df_gold_concat.iloc[:, 2]*profit_pct,
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='MA 2m (lower)',
        x=df_gold_concat.index,
        y=df_gold_concat.iloc[:, 2]-df_gold_concat.iloc[:, 2]*profit_pct,
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ),
    go.Scatter(
        name='Sell',
        x=df_gold_concat.index,
        y=df_gold_concat.iloc[:, 0],
        mode='lines',
        line=dict(color='rgb(34,139,34)'),
    ),
    go.Scatter(
        name='Buy',
        x=df_gold_concat.index,
        y=df_gold_concat.iloc[:, 1],
        mode='lines',
        line=dict(color='rgb(178,34,34)'),
    )
])
fig.update_layout(
    yaxis_title='Price in Rupiah',
    title='Gold [2019-05-17 : NOW]',
    hovermode="x"
)
fig.show()

In [ ]:
df_result = df_gold_concat.loc['2021-01-01':]

fig = go.Figure([
    go.Scatter(
        name='MA 1y',
        x=df_result.index,
        y=df_result.iloc[:, 3],
        mode='lines',
        line=dict(color='rgb(255,127,80)'),
    ),
    go.Scatter(
        name='MA 2m',
        x=df_result.index,
        y=df_result.iloc[:, 2],
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
    ),
    go.Scatter(
        name='MA 2m (upper)',
        x=df_result.index,
        y=df_result.iloc[:, 2]+df_result.iloc[:, 2]*profit_pct,
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='MA 2m (lower)',
        x=df_result.index,
        y=df_result.iloc[:, 2]-df_result.iloc[:, 2]*profit_pct,
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ),
    go.Scatter(
        name='Sell',
        x=df_result.index,
        y=df_result.iloc[:, 0],
        mode='lines',
        line=dict(color='rgb(34,139,34)'),
    ),
    go.Scatter(
        name='Buy',
        x=df_result.index,
        y=df_result.iloc[:, 1],
        mode='lines',
        line=dict(color='rgb(178,34,34)'),
    )
])
fig.update_layout(
    yaxis_title='Price in Rupiah',
    title='Gold [2021-01-01 : NOW]',
    hovermode="x"
)
fig.show()

In [ ]:
print(np.datetime64(df_gold_concat.index.values[-1], 'D'))
print()

gold_price_buy = df_gold_concat.iloc[-1, 0]
gold_price_sell = df_gold_concat.iloc[-1, 1]
gold_MA_2m = df_gold_concat.iloc[-1, 2]

if gold_MA_2m > gold_price_buy:
    print("It looks like gold price is still under value despite of it's volatility!")
    print('buy!')
elif gold_MA_2m + gold_MA_2m*0.1 < gold_price_sell:
    print("The price has passed the upper threshold in 10% margin range, time to sell!")
    print('sell!')
else:
    print("Gold price currently in steady-state, let's wait and see!")
    print('hold!')

In [ ]:
df_gold_logammulia = df['logammuliacom-gold-daily-dataset.csv']

# df_gold_logammulia['unixtime'] = df['logammuliacom-gold-daily-dataset.csv']['unixtime'].map(lambda x: str(x)[:-3])
# df_gold_logammulia['datetime'] = df_gold_logammulia['unixtime'].map(lambda x: datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))
df_gold_logammulia['datetime'] = pd.to_datetime(df_gold_logammulia['datetime'], format='%Y-%m-%d %H:%M')
df_gold_logammulia = df_gold_logammulia.set_index('datetime')

df_result = df_gold_logammulia
fig = go.Figure([
    go.Scatter(
        name='MA 1y',
        x=df_result.index,
        y=df_result.price,
        mode='lines',
        line=dict(color='rgb(255,127,80)'),
    )
])
fig.update_layout(
    yaxis_title='Price in Rupiah',
    title='Gold [2010-01-04 : NOW]',
    hovermode="x"
)
fig.show()

In [ ]:
start = datetime(2017, 1, 5)
start1 = start + timedelta(days = 1)
end = start + timedelta(days = 362)
end1 = end + timedelta(days = 1)

data_0 = df_gold_logammulia.loc[start:start1]
data_0 = data_0.iloc[0].price
data_1 = df_gold_logammulia.loc[end:end1]
data_1 = data_1.iloc[0].price

print(((data_1-data_0)/data_0)*100)

df_result = df_gold_logammulia.loc[start:end1]
fig = go.Figure([
    go.Scatter(
        name='MA 1y',
        x=df_result.index,
        y=df_result.price,
        mode='lines',
        line=dict(color='rgb(255,127,80)'),
    )
])
fig.update_layout(
    yaxis_title='Price in Rupiah',
    title='Gold [2010-01-04 : NOW]',
    hovermode="x"
)
fig.show()

In [ ]:
fig = go.Figure([
    go.Scatter(
        name='MA 1y',
        x=df_gold_logammulia.index,
        y=df_gold_logammulia.price,
        mode='lines',
        line=dict(color='rgb(255,127,80)'),
    )
])
fig.update_layout(
    yaxis_title='Price in Rupiah',
    title='Gold [2010-01-04 : NOW]',
    hovermode="x"
)
fig.show()

Let's be clear, I think if we want to gain some Return of Investment in a very short amount of time, looks like Gold is not an option. If we see the data since 2010, it's in a very stable state that steadily increasing 6% of return yearly. I have to say, if you got alot of free money in your pocket and you don't want to take any risk in your investment, Gold is a good option due to its steady-state increament and it still above yearly inflation.

In [ ]:
df_fed_funds_rate = df['macrotrends-fed-funds-rate-historical-chart.csv']
df_fed_funds_rate['date'] = pd.to_datetime(df_fed_funds_rate['date'], format='%Y-%m-%d')
df_fed_funds_rate = df_fed_funds_rate.set_index('date')

df_wsj_snp500 = df['wsj-index-spx-historical-prices.csv']
df_wsj_snp500['Date'] = pd.to_datetime(df_wsj_snp500['Date'], format='%m/%d/%y')
df_wsj_snp500 = df_wsj_snp500.set_index('Date')

In [ ]:
fig = go.Figure([
    go.Scatter(
        name='Fed Funds Rate',
        x=df_fed_funds_rate.index,
        y=df_fed_funds_rate.iloc[:, 0],
        mode='lines',
        line=dict(color='rgb(255,127,80)'),
    )
])
fig.update_layout(
    yaxis_title='index in percentage',
    title='Fed Funds Rate',
    hovermode="x"
)
fig.show()

In [ ]:
df_result = df_fed_funds_rate.loc[datetime(1990, 1, 1):datetime(2021, 5, 5)]

fig = go.Figure([
    go.Scatter(
        name='Fed Funds Rate',
        x=df_result.index,
        y=df_result.iloc[:, 0],
        mode='lines',
        line=dict(color='rgb(255,127,80)'),
    )
])
fig.update_layout(
    yaxis_title='index in percentage',
    title='Fed Funds Rate',
    hovermode="x"
)
fig.show()

In [ ]:
df_wsj_snp500 = df['wsj-index-spx-historical-prices.csv']
df_wsj_snp500['Date'] = pd.to_datetime(df_wsj_snp500['Date'], format='%m/%d/%y')
df_wsj_snp500 = df_wsj_snp500.set_index('Date')

In [ ]:
df_us_gdp = df['stlouisfed-us-gdp.csv']
df_us_gdp['DATE'] = pd.to_datetime(df_us_gdp['DATE'], format='%Y-%m-%d')
df_us_gdp = df_us_gdp.set_index('DATE')

df_us_gdp.info()
df_us_gdp

In [ ]:
df_us_m2 = df['stlouisfed-us-m2.csv']
df_us_m2['DATE'] = pd.to_datetime(df_us_m2['DATE'], format='%Y-%m-%d')
df_us_m2 = df_us_m2.set_index('DATE')

df_us_m2.info()
df_us_m2

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(
        name='SNP500 Index',
        x=df_wsj_snp500.index,
        y=df_wsj_snp500.iloc[:, 0],
        mode='lines',
        line=dict(color='rgb(255,127,80)'),
    ),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(
        name='Fed Funds Rate',
        x=df_fed_funds_rate.index,
        y=df_fed_funds_rate.iloc[:, 0],
        mode='lines',
        line=dict(color='rgb(70,130,180)'),
    ),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Double Y Axis Example"
)

# Set x-axis title
fig.update_xaxes(title_text="xaxis title")

# Set y-axes titles
fig.update_yaxes(title_text="index", secondary_y=False)
fig.update_yaxes(title_text="%", secondary_y=True)

fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_wsj_snp500.index,
    y=df_wsj_snp500.iloc[:, 0],
    name="SNP500"
))


fig.add_trace(go.Scatter(
    x=df_fed_funds_rate.index,
    y=df_fed_funds_rate.iloc[:, 0],
    name="Fed Funds Rate",
    yaxis="y2"
))

fig.add_trace(go.Scatter(
    x=df_us_gdp.index,
    y=df_us_gdp.iloc[:, 0],
    name="US GDP",
    yaxis="y3"
))

fig.add_trace(go.Scatter(
    x=df_us_m2.index,
    y=df_us_m2.iloc[:, 0],
    name="US M2",
    yaxis="y4"
))


# Create axis objects
fig.update_layout(
    xaxis=dict(
        domain=[0.07, 0.93]
    ),
    yaxis=dict(
        title="SNP500",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        )
    ),
    yaxis2=dict(
        title="Fed Funds Rate",
        titlefont=dict(
            color="#ff7f0e"
        ),
        tickfont=dict(
            color="#ff7f0e"
        ),
        anchor="free",
        overlaying="y",
        side="left",
        position=0
    ),
    yaxis3=dict(
        title="US GDP",
        titlefont=dict(
            color="#d62728"
        ),
        tickfont=dict(
            color="#d62728"
        ),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    yaxis4=dict(
        title="US M2",
        titlefont=dict(
            color="#9467bd"
        ),
        tickfont=dict(
            color="#9467bd"
        ),
        anchor="free",
        overlaying="y",
        side="right",
        position=1
    )
)

# Update layout properties
fig.update_layout(
    title_text="multiple y-axes example",
#     width=1200,
)

fig.show()

In [ ]:
fig = go.Figure([
    go.Scatter(
        name='MA 1y',
        x=df_fed_funds_rate.index,
        y=df_fed_funds_rate.iloc[:, 0],
        mode='lines',
        line=dict(color='rgb(255,127,80)'),
    )
])
fig.update_layout(
    yaxis_title='index in percentage',
    title='Fed Funds Rate',
    hovermode="x"
)
fig.show()

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(
        name='MA 1y',
        x=df_gold_logammulia.index,
        y=df_gold_logammulia.price,
        mode='lines',
        line=dict(color='rgb(255,127,80)'),
    ),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(
        name='MA 1y',
        x=df_fed_funds_rate.index,
        y=df_fed_funds_rate.iloc[:, 0],
        mode='lines',
        line=dict(color='rgb(70,130,180)'),
    ),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Double Y Axis Example"
)

# Set x-axis title
fig.update_xaxes(title_text="xaxis title")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> yaxis title", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)

fig.show()

Bitcoin

In [ ]:
df['pluang-bitcoin-daily-dataset.csv']['priceStatDate'] = pd.to_datetime(df['pluang-bitcoin-daily-dataset.csv']['priceStatDate'])
df_btc_pivot = df['pluang-bitcoin-daily-dataset.csv'].pivot(index='priceStatDate', columns='currency', values=['closeSell'])

In [ ]:
df_btc_pivot.plot(figsize=(25,13), grid=True)

In [ ]:
df_btc_pivot.loc['2021-01-01':].plot(figsize=(25,13), grid=True)

In [ ]:
df_btc_growth = df_btc_pivot
df_btc_growth['growth'] = df_btc_pivot['closeSell'].pct_change().mul(100).round(2)
df_gold_growth = df_gold_concat
df_gold_growth['sell (growth)'] = df_gold_concat.iloc[:,0].pct_change().mul(100).round(2)

In [ ]:
df_concat = pd.concat([df_gold_growth, df_btc_growth.loc['2019-05-17':]], axis=1)
df_concat.iloc[:,0] = df_concat.iloc[:,0].ffill()
df_concat.iloc[:,3] = df_concat.iloc[:,0].pct_change().mul(100).round(2)
df_concat.iloc[:,4] = df_concat.iloc[:,4].ffill()
df_concat.iloc[:,5] = df_concat.iloc[:,4].pct_change().mul(100).round(2)

In [ ]:
df_concat.iloc[:,3].plot(figsize=(25,13), grid=True)

In [ ]:
start_date = '2021-01-01'

plt.figure(figsize=(25,13))
plt.xlabel('Gold growth vs BTC growth')

ax1 = df_concat.iloc[:,3].loc[start_date:].plot(color='blue', grid=True, label='Gold (growth)')
ax2 = df_concat.iloc[:,5].loc[start_date:].plot(color='red', grid=True, label='Bitcoin (growth)')

h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()

plt.legend(h1+h2, l1+l2, loc=2)
plt.show()

In [ ]:
df_concat_monthly = df_concat.resample('1M').last()
df_concat_monthly.iloc[:,3] = df_concat_monthly.iloc[:,0].pct_change().mul(100).round(2)
df_concat_monthly.iloc[:,5] = df_concat_monthly.iloc[:,4].pct_change().mul(100).round(2)

In [ ]:
start_date = '2019-05-31'

plt.figure(figsize=(25,13))
plt.xlabel('Gold growth vs BTC growth')

ax1 = df_concat_monthly.iloc[:,3].loc[start_date:].plot(color='blue', grid=True, label='Gold (growth)')
ax2 = df_concat_monthly.iloc[:,5].loc[start_date:].plot(color='red', grid=True, label='Bitcoin (growth)')

h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()

plt.legend(h1+h2, l1+l2, loc=2)
plt.show()

In [ ]:
df_concat_monthly = df_concat.resample('1M').mean()
df_concat_monthly.iloc[:,3] = df_concat_monthly.iloc[:,0].pct_change().mul(100).round(2)
df_concat_monthly.iloc[:,5] = df_concat_monthly.iloc[:,4].pct_change().mul(100).round(2)

In [ ]:
start_date = '2019-05-31'

plt.figure(figsize=(25,13))
plt.xlabel('Gold growth vs BTC growth')

ax1 = df_concat_monthly.iloc[:,3].loc[start_date:].plot(color='blue', grid=True, label='Gold (growth)')
ax2 = df_concat_monthly.iloc[:,5].loc[start_date:].plot(color='red', grid=True, label='Bitcoin (growth)')

h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()

plt.legend(h1+h2, l1+l2, loc=2)
plt.show()